### Usage Histogram Interactive Plot

similar in usage to: https://github.com/bokeh/bokeh/blob/master/examples/app/selection_histogram.py

In [1]:
import pandas as pd
import numpy as np

In [2]:
import gzip
import pickle

This is the library data processed in the percent usage per hour.

In [3]:
with gzip.open(r'../data/LibData.pkl.gz') as f:
    libraryData = pickle.load(f)

The dates here show the data from 3/24/10 - 10/19/17

In [4]:
libraryData.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 66381 entries, 2010-03-24 12:00:00 to 2017-10-19 08:00:00
Freq: H
Columns: 312 entries, BL001 to TL902
dtypes: float64(312)
memory usage: 158.5 MB


This is a grouping of the library data by average per hour using arbitrary endpoints.

In [5]:
startDate = pd.to_datetime("2016-01-01")
endDate = pd.to_datetime("2017-12-31")
dateMask = (libraryData.index > startDate) & (libraryData.index < endDate)

The computer attributes need to be loaded into a separate dataframe:

In [6]:
compAttrs = pd.read_csv(r'../data/computerAttributes.csv',header=0)

In [7]:
compAttrs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 312 entries, 0 to 311
Data columns (total 14 columns):
dbID                  312 non-null int64
computerName          312 non-null object
requiresLogon         311 non-null float64
isDesktop             311 non-null float64
inJackson             311 non-null float64
location              306 non-null object
is245                 311 non-null float64
floor                 245 non-null object
numMonitors           304 non-null float64
largeMonitor          238 non-null float64
adjacentWindow        238 non-null float64
collaborativeSpace    238 non-null float64
roomIsolated          238 non-null float64
inQuietArea           238 non-null float64
dtypes: float64(10), int64(1), object(3)
memory usage: 34.2+ KB


In [8]:
booleanCols = ["requiresLogon",
               "isDesktop",
               "inJackson",
               "is245",
               "largeMonitor",
               "adjacentWindow",
               "collaborativeSpace",
               "roomIsolated",
               "inQuietArea"]

Using the attributes from above as booleans, create a mask for the `compAttrs` dataframe, and return the names.

In [9]:
attrsNames = compAttrs[compAttrs.requiresLogon == True].computerName

In [10]:
libraryMeans = libraryData[dateMask].groupby(libraryData[dateMask].index.hour).mean()*100

In [11]:
libraryMeans.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24 entries, 0 to 23
Columns: 312 entries, BL001 to TL902
dtypes: float64(312)
memory usage: 58.7 KB


Since the format is a 24 (hours) x 312 (computers) matrix, and the scatter plot is looking for single-dimension arrays, the data needs to be unstacked into these arrays.

In [12]:
meansUnstacked = libraryMeans.unstack().reset_index()
meansUnstacked.columns = ["comps","hour","means"]

In [13]:
meansUnstackedMerged = meansUnstacked.merge(compAttrs, left_on='comps',right_on='computerName')

Since the number of machines in the later graph might change, going ahead here and setting variables based on the count of machines returned in the dataframe above:

In [14]:
machineCount = meansUnstackedMerged.comps.unique().size
recordCount = meansUnstackedMerged.index.size
hourCount = 24

The Bokeh libraries necessary for this graph:

In [15]:
from bokeh.layouts import row, column
from bokeh.models import BoxSelectTool, LassoSelectTool, Spacer, FuncTickFormatter, FixedTicker, HoverTool, ColumnDataSource
from bokeh.plotting import figure, output_file, output_notebook, show, save
output_notebook()

Loading BokehJS ...

Bokeh allows a number of tools included in the tool bar adjacent to the graph. Testing the tools available and configurations for each. Hover, in this case, uses the data from the ColumnDataSource to populate the tooltips.

In [16]:
hover = HoverTool(
    tooltips=[
        ("Computer", "@comps"),
        ("Hour", "$y{0}:00"),
        ("Pct Use","@means")
    ],
    formatters={"Hour":"datetime"}
)
#TOOLS=[hover,"crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select"]
TOOLS=[hover,"crosshair,pan,wheel_zoom,box_zoom,reset,tap,save,box_select,poly_select,lasso_select"]

This ColumnDataSource is necessary to pass the dataframe values to the scatter plot later.

In [17]:
source = ColumnDataSource.from_df(meansUnstackedMerged)

These are the basic commands to create the graph known as `mainGraph`. The `select()` commands are perceived to improve performance on large datasets

In [18]:
mainGraph = figure(tools=TOOLS, plot_width=900, plot_height=600,
                     min_border=10, min_border_left=50,
                     toolbar_location="above",
                     x_axis_location=None, # this is left in, as the x-axis ticks are hard to read zoomed out.
                     #y_axis_location=None, 
                     title="Library Usage: Average Percent Utilization per Hour")
mainGraph.background_fill_color = "#fafafa"
mainGraph.select(BoxSelectTool).select_every_mousemove = False
mainGraph.select(LassoSelectTool).select_every_mousemove = False

Formatting the tickers requires some finesse. This first example uses some JavaScript to format the tick values to a 24-hour clock, and then constrain it to integers.

In [19]:
mainGraph.yaxis.formatter = FuncTickFormatter(code="""return Math.floor(tick)+':00'""")
mainGraph.yaxis.ticker = FixedTicker(ticks = range(0,24))

Formatting the xaxis requires aligning the computernames to the values within the unstacked dataframe. This is omitted for now. 

In [20]:
keys=range(0,recordCount,hourCount)
values=list(meansUnstackedMerged.comps.unique())
graphCompIndex = dict(zip(keys,values))
mainGraph.xaxis.ticker = FixedTicker(ticks = range(0,recordCount,hourCount))
mainGraph.xaxis.major_label_overrides = graphCompIndex

In [22]:
mainGraph.scatter("index","hour",radius="means",color="blue",alpha=.4,source=source)
#output_file("./AvgPercentUtil.html", title='Library Usage: Average Percent Utilization per Hour')
show(mainGraph)